In [1]:
import pandas as pd
import json

In [2]:
df = pd.read_csv('data/Payments to US Entities Working - Entities - Clean.csv')

In [3]:
df.head()

,Entity Name,Reporting Company,Payment Type,Start Date,End Date,Federal Government,Tribal Governments,State,Value (USD),Notes,Unnamed: 10
0,State of Alaska,Agnico Eagle Mines Limited,Fees,01-01-2016,12-31-2016,N,NaN,Alaska,75448.0,Department of Natural Resources,NaN
1,United States Government,Agnico Eagle Mines Limited,Fees,01-01-2016,12-31-2016,Y,NaN,District of Columbia,377244.0,Bureau of Land Management,NaN
2,Arizona Department of Revenue,Agrium Inc.,Taxes,01-01-2016,12-31-2016,N,NaN,Arizona,347065.0,State taxes paid - $348K USD translated at an ...,NaN
3,State of California,Agrium Inc.,Taxes,01-01-2016,12-31-2016,N,NaN,California,6224537.0,State taxes paid - $6.19M USD translated at an...,NaN
4,Colorado Department of Revenue,Agrium Inc.,Taxes,01-01-2016,12-31-2016,N,NaN,Colorado,347065.0,State taxes paid - $343K USD translated at an ...,NaN


In [4]:
df["Government Type"] = "State"
df.loc[df["Tribal Governments"].notnull(),"Government Type"] = "Tribal"
df.loc[df["Federal Government"] == "Y","Government Type"] = "Federal"
df.loc[df["State"].isnull(),"Government Type"] = "Unknown"

In [5]:
df.loc[df["State"].isnull()]

,Entity Name,Reporting Company,Payment Type,Start Date,End Date,Federal Government,Tribal Governments,State,Value (USD),Notes,Unnamed: 10,Government Type
52,Various counties,Baytex Energy Corp.,Taxes,01-01-2016,12-31-2016,N,NaN,NaN,4632565.0,Exchange rate from USD to CAD used in this rep...,NaN,Unknown
247,County Recorder (Various),Eurasian Minerals Inc.,Fees,01-01-2016,12-31-2016,N,NaN,NaN,10799.0,County recording fees,NaN,Unknown
248,Secretary of State (Various),Eurasian Minerals Inc.,Fees,01-01-2016,12-31-2016,NaN,NaN,NaN,1084.0,Filing fees,NaN,Unknown
365,COMMISSIONER OF THE GENERAL LAND,ROYAL DUTCH SHELL PLC,Fees,01-01-2016,12-31-2016,NaN,NaN,NaN,203100.0,NaN,There is a commissioner of the general land in...,Unknown
366,"DIRECTOR, DIVISION OF OIL AND GAS",ROYAL DUTCH SHELL PLC,Fees,01-01-2016,12-31-2016,NaN,NaN,NaN,243408.0,NaN,"Again, most states have a division of oil and gas",Unknown
380,State Emergency Response Commission,Rye Patch Gold Corp.,Fees,01-01-2016,12-31-2016,N,NaN,NaN,3772.0,USD payments translated at 2016 annual average...,NaN,Unknown
442,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown
443,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown
444,Note: Removed any company w/ fiscal year endin...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown


In [6]:
df = df.drop(df.index[442:445])

In [7]:
df = df[df["Value (USD)"] >= 0]

## Prepare Source-Target-Value dataframe

In [8]:
links = pd.DataFrame(columns=['source','target','value'])


In [9]:
to_append = df.groupby(['Government Type'],as_index=False)['Value (USD)'].sum()
to_append["source"] = "Total"
to_append.rename(columns = {'Government Type':'target', 'Value (USD)' : 'value'}, inplace = True)
to_append = to_append.sort_values(by=['value'], ascending = False)
links = pd.concat([links,to_append])

print(to_append['value'].sum())
links

2360235274.0


,source,target,value
0,Total,Federal,1.326789e+09
1,Total,State,9.593532e+08
2,Total,Tribal,6.899835e+07
3,Total,Unknown,5.094728e+06


In [10]:
to_append = df.groupby(['Government Type','Payment Type'],as_index=False)['Value (USD)'].sum()
to_append.rename(columns = {'Government Type':'source','Payment Type':'target', 'Value (USD)' : 'value'}, inplace = True)
to_append = to_append.sort_values(by=['value'], ascending = False)
links = pd.concat([links,to_append])

print(to_append['value'].sum())
links

2360235274.0


,source,target,value
0,Total,Federal,1.326789e+09
1,Total,State,9.593532e+08
2,Total,Tribal,6.899835e+07
3,Total,Unknown,5.094728e+06
2,Federal,Royalties,7.944091e+08
8,State,Taxes,5.901037e+08
3,Federal,Taxes,4.096844e+08
7,State,Royalties,2.787502e+08
1,Federal,Fees,9.466987e+07
5,State,Fees,8.414347e+07


In [11]:
to_append = df.groupby(['Payment Type','Reporting Company'],as_index=False)['Value (USD)'].sum()
to_append.rename(columns = {'Payment Type':'source','Reporting Company':'target', 'Value (USD)' : 'value'}, inplace = True)
to_append = to_append.sort_values(by=['value'], ascending = False)
links = pd.concat([links,to_append])

print(to_append['value'].sum())
links

2360235274.0


,source,target,value
0,Total,Federal,1.326789e+09
1,Total,State,9.593532e+08
2,Total,Tribal,6.899835e+07
3,Total,Unknown,5.094728e+06
2,Federal,Royalties,7.944091e+08
8,State,Taxes,5.901037e+08
3,Federal,Taxes,4.096844e+08
7,State,Royalties,2.787502e+08
1,Federal,Fees,9.466987e+07
5,State,Fees,8.414347e+07


In [12]:
unique_list = pd.concat([links['source'], links['target']]).unique()
replace_dict = {k: v for v, k in enumerate(unique_list)}


In [13]:
links_replaced = links.replace({"source": replace_dict,"target": replace_dict})

In [14]:
links_replaced

,source,target,value
0,0,1,1.326789e+09
1,0,2,9.593532e+08
2,0,3,6.899835e+07
3,0,4,5.094728e+06
2,1,5,7.944091e+08
8,2,6,5.901037e+08
3,1,6,4.096844e+08
7,2,5,2.787502e+08
1,1,7,9.466987e+07
5,2,7,8.414347e+07


In [15]:
nodes = pd.DataFrame(unique_list)
nodes.rename(columns = {0:'name'}, inplace = True)

In [16]:
nodes_json= pd.DataFrame(nodes).to_json(orient='records')
nodes_json 

'[{"name":"Total"},{"name":"Federal"},{"name":"State"},{"name":"Tribal"},{"name":"Unknown"},{"name":"Royalties"},{"name":"Taxes"},{"name":"Fees"},{"name":"Production entitlements"},{"name":"Bonuses"},{"name":"Payments for infrastructure improvements"},{"name":"BP P.L.C."},{"name":"ROYAL DUTCH SHELL PLC"},{"name":"Encana Corporation"},{"name":"Barrick Gold Corporation"},{"name":"Agrium Inc."},{"name":"Teck Resources Limited"},{"name":"Rio Tinto PLC"},{"name":"Enerplus Corporation"},{"name":"Kinross Gold Corporation"},{"name":"CNOOC Limited"},{"name":"Potash Corporation of Saskatchewan Inc."},{"name":"Silver Standard Resources Inc."},{"name":"Graymont Limited"},{"name":"Coeur Mining, Inc."},{"name":"Baytex Energy Corp."},{"name":"Goldcorp Inc."},{"name":"Crescent Point Energy Corp."},{"name":"Capstone Mining Corp."},{"name":"PetroShale Inc."},{"name":"Energy Fuels Inc"},{"name":"Vermilion Energy Inc."},{"name":"Cameco Corporation"},{"name":"Votorantim Cement North America Inc."},{"name":

In [17]:
links_json= pd.DataFrame(links_replaced).to_json(orient='records')
links_json 

'[{"source":0,"target":1,"value":1326789026.0},{"source":0,"target":2,"value":959353173.0},{"source":0,"target":3,"value":68998347.0},{"source":0,"target":4,"value":5094728.0},{"source":1,"target":5,"value":794409147.0},{"source":2,"target":6,"value":590103721.0},{"source":1,"target":6,"value":409684392.0},{"source":2,"target":5,"value":278750235.0},{"source":1,"target":7,"value":94669871.0},{"source":2,"target":7,"value":84143470.0},{"source":3,"target":8,"value":31955077.0},{"source":1,"target":9,"value":28025616.0},{"source":3,"target":5,"value":22103776.0},{"source":3,"target":6,"value":7431555.0},{"source":2,"target":9,"value":6265747.0},{"source":3,"target":7,"value":5229382.0},{"source":4,"target":6,"value":4632565.0},{"source":3,"target":9,"value":2278557.0},{"source":4,"target":7,"value":462163.0},{"source":2,"target":10,"value":90000.0},{"source":5,"target":11,"value":602620019.0},{"source":5,"target":12,"value":374777662.0},{"source":6,"target":13,"value":191680000.0},{"sour

In [18]:
data = { 'links' : json.loads(links_json), 'nodes' : json.loads(nodes_json) }
data_json = json.dumps(data)
data_json = data_json.replace("\\","")
#print(data_json)
#with open('sankey_data.json', 'w') as outfile:
#    json.dump(data_json, outfile)

text_file = open("sankey_data.json", "w")
text_file.write(data_json)
text_file.close()